In [4]:
import sys

sys.path.append("../")

In [2]:
import boto3
import io
import pandas as pd

s3 = boto3.client("s3")
obj = s3.get_object(Bucket="ds-rg271", Key="data/labelled/stratified_random_sampling/test.csv")
df = pd.read_csv(io.BytesIO(obj['Body'].read()))
df = df.loc[df["complaint"]]

In [5]:
from RG271.binary_cls.inference import InferenceModel
from pathlib import Path

if not Path("model.pkl").exists():
    s3.download_file("ds-rg271", "models/stratified_random_sampling/model.pkl", "model.pkl")
    
model = InferenceModel("model.pkl")

In [8]:
for row in df.itertuples():
    output = model.predict(row.content)
    df.at[row.Index, "prediction"] = output["label_id"]

In [10]:
df["prediction"].astype(int)

0    0
1    1
0    0
0    0
1    1
    ..
1    1
1    1
0    0
1    1
0    0
Name: prediction, Length: 173, dtype: int64

In [11]:
from sklearn.metrics import confusion_matrix

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    return {
        "tp": tp,
        "fp": fp,
        "tn": tn,
        "fn": fn,
        "accuracy": (tn + tp) / (tn + fp + fn + tp),
        "precision": precision,
        "recall": recall,
        "f1": 2 * (precision * recall) / (precision + recall),
    }

calculate_metrics(df["complaint"], df["prediction"])

{'tp': 63,
 'fp': 0,
 'tn': 110,
 'fn': 0,
 'accuracy': 1.0,
 'precision': 1.0,
 'recall': 1.0,
 'f1': 1.0}